In [12]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import time
from pandas import DataFrame
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report, precision_score,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
standardsc = StandardScaler()
mas = MaxAbsScaler()

In [2]:
tcga = pd.read_csv('TCGA_data.csv')
tcga.drop('cancer', axis = 1, inplace = True)
tcga = tcga.sort_index(axis=1)
tcga['y'] = tcga['y'].replace(['KIRC','KIRP','KICH'],['RC','RC','RC'])

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
tcga["y_encode"] = lb_make.fit_transform(tcga["y"])

In [10]:
urine = pd.read_csv('urine_k15_0419.csv')
urine.drop(['Unnamed: 0','X'],axis=1,inplace=True)
urine = urine.sort_index(axis=1)
urine.y.value_counts()

BLCA        51
NL          28
PRAD        23
RCC_100k    21
Name: y, dtype: int64

In [11]:
urine = urine.replace(['BLCA','NL','PRAD','RCC_100k'],[ 0,12,19,17])
urine = urine.reset_index(drop = True)
print('Urine data shape:',urine.shape)
urine.y.value_counts()

Urine data shape: (123, 787)


0     51
12    28
19    23
17    21
Name: y, dtype: int64

In [14]:
X = tcga.drop(['y','y_encode'], axis=1)
y = tcga['y_encode']

standardsc.fit(X)
X_standar = standardsc.transform(X)
urine_standar = standardsc.transform(urine.drop('y', axis = 1))

mas.fit(X)
X_mas = standardsc.transform(X)
urine_mas = mas.transform(urine.drop('y', axis = 1))

In [5]:
def pred_accuracy(model) :
    pred = model.predict(urine.drop('y', axis = 1))
    df_pred = pd.DataFrame({'actual':urine['y'],'predict':pred})
    
    BC = df_pred.loc[df_pred['actual']==0]
    NL = df_pred.loc[df_pred['actual']==15]
    PC = df_pred.loc[df_pred['actual']==19]
    
    BC_pred = BC.loc[BC['predict']==0]
    NL_pred = NL.loc[NL['predict']==15]
    PC_pred = PC.loc[PC['predict']==19]
    
    print('BC accuracy : {0:.4f}'.format(BC_pred.shape[0]/BC.shape[0]))
    print('Normal accuracy : {0:.4f}'.format(NL_pred.shape[0]/NL.shape[0]))
    print('PC accuracy : {0:.4f}'.format(PC_pred.shape[0]/PC.shape[0]))

# Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 0, n_jobs = -1, max_depth=75,n_estimators=500)
rf.fit(X_mas,y)
pred = rf.predict(urine_mas)
accuracy = accuracy_score(urine['y'],pred)
print('RF accuracy : {0:.4f}'.format(accuracy))

sec : 45.42
RF accuracy : 0.0407


In [17]:
rf = RandomForestClassifier(random_state = 0, n_jobs = -1, max_depth=75,n_estimators=500)
rf.fit(X,y)
pred = rf.predict(urine.drop('y', axis = 1))
accuracy = accuracy_score(urine['y'],pred)
print('RF accuracy : {0:.4f}'.format(accuracy))

RF accuracy : 0.1545


# Logistic

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler
from sklearn.linear_model import LogisticRegression

standardsc = StandardScaler()
mms = MinMaxScaler()
mas = MaxAbsScaler()

lr = LogisticRegression()

In [12]:
def model_pipeline(model) :
    pipeline1 = Pipeline([('StandardScaler', standardsc), ('Logistic', model)])
    pipeline2 = Pipeline([('MinMaxScaler', mms), ('Logistic', model)])
    pipeline3 = Pipeline([('MaxAbsScaler', mas), ('Logistic', model)])

    cv_acc = []
    pip1_acc = []
    pip2_acc = []
    pip3_acc = []

    for i, (tr_ind, te_ind) in enumerate(skf.split(X,y)) :

        X_train, X_test = X.iloc[tr_ind], X.iloc[te_ind]
        y_train, y_test = y[tr_ind], y[te_ind]

        model.fit(X_train, y_train)
        pipeline1.fit(X_train, y_train)
        pipeline2.fit(X_train, y_train)
        pipeline3.fit(X_train, y_train)

        cv_acc.append(accuracy_score(y[te_ind], model.predict(X_test)))
        pip1_acc.append(accuracy_score(y[te_ind], pipeline1.predict(X_test)))
        pip2_acc.append(accuracy_score(y[te_ind], pipeline2.predict(X_test)))
        pip3_acc.append(accuracy_score(y[te_ind], pipeline3.predict(X_test)))

        print('{0} 번째 accuracy non_scale : {1:.4f}% StandardScale : {2:.4f}% MinMax : {3:.4f}% MaxAbs : {4:.4f}%'.format(i,cv_acc[i],pip1_acc[i],pip2_acc[i],pip3_acc[i]))

    print('\n mean accuracy non_scale : {0:.4f}% StandardScale : {1:.4f}% MinMax : {2:.4f}% MaxAbs : {3:.4f}%'.format(np.mean(cv_acc),np.mean(pip1_acc),np.mean(pip2_acc),np.mean(pip3_acc)))

In [ ]:
model_pipeline(lr)

0 번째 accuracy non_scale : 0.8048% StandardScale : 0.7896% MinMax : 0.5000% MaxAbs : 0.7944%
1 번째 accuracy non_scale : 0.7990% StandardScale : 0.7821% MinMax : 0.4958% MaxAbs : 0.7886%
2 번째 accuracy non_scale : 0.7999% StandardScale : 0.7801% MinMax : 0.4975% MaxAbs : 0.7929%
3 번째 accuracy non_scale : 0.8035% StandardScale : 0.7786% MinMax : 0.4978% MaxAbs : 0.7876%


In [7]:
from sklearn.linear_model import LogisticRegression
start_time = time.time()
lr = LogisticRegression(n_jobs=-1)
lr.fit(X,y)
print('sec : ',time.time()-start_time) # 2 minutes

sec :  108.8355324268341


In [8]:
pred = lr.predict(urine.drop('y', axis = 1))
accuracy = accuracy_score(urine['y'],pred)
print('Logistic accuracy : {0:.4f}'.format(accuracy))

Logistic accuracy : 0.3253


In [9]:
pred_accuracy(lr)

BC accuracy : 0.1190
Normal accuracy : 1.0000
PC accuracy : 0.0500


In [11]:
pred = lr.predict(urine.drop('y', axis = 1))
df_pred = pd.DataFrame({'actual':urine['y'],'predict':pred})


In [23]:
df_pred[df_pred['actual']==0].predict.value_counts()/len(df_pred[df_pred['actual']==0])

15    0.309524
0     0.119048
5     0.095238
27    0.071429
19    0.071429
13    0.071429
2     0.071429
25    0.047619
11    0.047619
1     0.047619
14    0.023810
10    0.023810
Name: predict, dtype: float64

In [17]:
df_pred[df_pred['actual']==15].predict.value_counts()

15    21
Name: predict, dtype: int64

In [24]:
df_pred[df_pred['actual']==19].predict.value_counts()/len(df_pred[df_pred['actual']==19])

15    0.60
5     0.10
3     0.10
27    0.05
19    0.05
11    0.05
1     0.05
Name: predict, dtype: float64

# LightGBM

In [15]:
from lightgbm import LGBMClassifier
start_time = time.time()

lgbm_clf = LGBMClassifier(n_estimators=200, n_jobs=-1)
lgbm_clf.fit(X, y, eval_metric="logloss", verbose=50)
print('sec : ',time.time()-start_time) # 4 minutes

sec :  236.48080492019653


In [16]:
pred = lgbm_clf.predict(urine.drop('y', axis = 1))
accuracy = accuracy_score(urine['y'], pred)
print('LightGBM accuracy : {0:.4f}'.format(accuracy))

LightGBM accuracy : 0.3373


In [17]:
pred_accuracy(lgbm_clf)

BC accuracy : 0.1667
Normal accuracy : 1.0000
PC accuracy : 0.0000
